In [1]:
# We need to install nightly package for some functionalities that aren't in alpha
!pip install tf-nightly-gpu-2.0-preview

In [2]:
# Mount your drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import warnings
warnings.filterwarnings("ignore")

import os
import cv2
import glob
import h5py
import shutil

import numpy as np 
import pandas as pd 
import seaborn as sns
from pathlib import Path
import matplotlib.pylab as pl
import matplotlib.pyplot as plt


from skimage.io import imread, imsave
from skimage.transform import resize
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.applications import resnet50, mobilenet, mobilenet_v2, vgg16, inception_v3
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Reshape
from tensorflow.keras.layers import Input, Flatten, Activation
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import top_k_categorical_accuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras import layers


color = sns.color_palette()
%matplotlib inline
%config InlineBackend.figure_format="svg"

In [0]:
# Set the seed for hash based operations in python
os.environ['PYTHONHASHSEED'] = '0'

seed=1234

# Set the numpy seed
np.random.seed(seed)

# Set the random seed in tensorflow at graph level
tf.random.set_seed(seed)

In [0]:
# Same constants as we used during modelling
img_height, img_width, img_channels = 224,224,3
batch_size = 64
nb_classes = 7

def top_2(y_true, y_pred):
    return top_k_categorical_accuracy(y_true=y_true, y_pred=y_pred, k=2)

In [8]:
# load the keras model
model = tf.keras.models.load_model("gdrive/My Drive/skin_cancer_data/skin_cancer_best_model.h5", custom_objects={'top_2': top_2})
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Model) (None, 1280)              2257984   
_________________________________________________________________
top_model (Model)            (None, 7)                 8967      
Total params: 2,266,951
Trainable params: 2,232,839
Non-trainable params: 34,112
_________________________________________________________________


## Converting tf.keras model to TFlite model

In [0]:
# convert the model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [0]:
# write the model to disk
f = open("skin_cancer_best_model.tflite", "wb")
f.write(tflite_model)
f.close()

In [0]:
# check if the model is saved correctly or not
f = open("skin_cancer_best_model.tflite", "rb")
tflite_model_cus = f.read()
f.close()

## Checking the converted tflite model 

In [0]:
# fire up a tflite interpreter
interpreter = tf.lite.Interpreter(model_content=tflite_model_cus)
interpreter.allocate_tensors()

In [0]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [14]:
# Check the inputs 
input_details

[{'dtype': numpy.float32,
  'index': 1,
  'name': 'input_2',
  'quantization': (0.0, 0),
  'shape': array([  1, 224, 224,   3], dtype=int32)}]

In [15]:
# Check the outputs 
output_details

[{'dtype': numpy.float32,
  'index': 0,
  'name': 'Identity',
  'quantization': (0.0, 0),
  'shape': array([1, 7], dtype=int32)}]

In [0]:
# Test the TensorFlow Lite model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

# Test the TensorFlow model on random input data.
tf_results = model(tf.constant(input_data))

# Compare the result.
for tf_result, tflite_result in zip(tf_results, tflite_results):
  np.testing.assert_almost_equal(tf_result, tflite_result, decimal=5)